In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/32 x 32 images/DCT_10.pkl', 'rb') as f:
    DCT_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/32 x 32 images/DCT_20.pkl', 'rb') as f:
    DCT_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/32 x 32 images/DCT_30.pkl', 'rb') as f:
    DCT_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/32 x 32 images/DCT_40.pkl', 'rb') as f:
    DCT_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_32x32_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
DCT_10_normalized = normalize_max_row_norm(DCT_10)
DCT_20_normalized = normalize_max_row_norm(DCT_20)
DCT_30_normalized = normalize_max_row_norm(DCT_30)
DCT_40_normalized = normalize_max_row_norm(DCT_40)

DCT_10_observations = generate_observations(img_dataset, DCT_10_normalized)
DCT_20_observations = generate_observations(img_dataset, DCT_20_normalized)
DCT_30_observations = generate_observations(img_dataset, DCT_30_normalized)
DCT_40_observations = generate_observations(img_dataset, DCT_40_normalized)

DCT_10_train, DCT_10_test = split_dataset(DCT_10_observations, train_ratio = 0.9, seed = 0)
DCT_20_train, DCT_20_test = split_dataset(DCT_20_observations, train_ratio = 0.9, seed = 0)
DCT_30_train, DCT_30_test = split_dataset(DCT_30_observations, train_ratio = 0.9, seed = 0)
DCT_40_train, DCT_40_test = split_dataset(DCT_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 10)

# Equal Weighting
loss_function = IntermediateScaledLoss(omega = 1.0)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'DCT_32_Depth10/5 Projections'

DCT_10_proj5_weights, DCT_10_proj5_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj5', display=True)

DCT_20_proj5_weights, DCT_20_proj5_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj5', display=True)

DCT_30_proj5_weights, DCT_30_proj5_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj5', display=True)

DCT_40_proj5_weights, DCT_40_proj5_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, DCT_10_proj5_weights, DCT_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, DCT_20_proj5_weights, DCT_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, DCT_30_proj5_weights, DCT_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, DCT_40_proj5_weights, DCT_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.13635461897320217
Average test loss: 0.005355825924625
Epoch 2/300
Average training loss: 0.032534917823142474
Average test loss: 0.005102198344965776
Epoch 3/300
Average training loss: 0.027109566436873542
Average test loss: 0.0049180238348328405
Epoch 4/300
Average training loss: 0.02523718679116832
Average test loss: 0.004668932194304135
Epoch 5/300
Average training loss: 0.024309474322530958
Average test loss: 0.004645234119147063
Epoch 6/300
Average training loss: 0.02377603591316276
Average test loss: 0.004551669690344069
Epoch 7/300
Average training loss: 0.02341022516290347
Average test loss: 0.004533002051628298
Epoch 8/300
Average training loss: 0.023137429507242308
Average test loss: 0.004506945891305804
Epoch 9/300
Average training loss: 0.022918275247017544
Average test loss: 0.004551655347769459
Epoch 10/300
Average training loss: 0.022734922339518866
Average test loss: 0.004444528242780103
Epoc

In [8]:
DCT_10_proj5_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj5_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj5_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj5_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj5_psnr = average_PSNR(DCT_10_proj5_model, DCT_10_proj5_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj5_psnr = average_PSNR(DCT_20_proj5_model, DCT_20_proj5_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj5_psnr = average_PSNR(DCT_30_proj5_model, DCT_30_proj5_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj5_psnr = average_PSNR(DCT_40_proj5_model, DCT_40_proj5_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj5_psnr, f)

/home/ubuntu/Memory-Net-Inverse/Denoising_Algorithms/DL_Training/evaluation.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(directory, 

Average PSNR for Projection Layer 0 across 2500 images: 27.85
Average PSNR for Projection Layer 1 across 2500 images: 28.20
Average PSNR for Projection Layer 2 across 2500 images: 28.43
Average PSNR for Projection Layer 3 across 2500 images: 28.61
Average PSNR for Projection Layer 4 across 2500 images: 28.69
Average PSNR for Projection Layer 0 across 2500 images: 28.40
Average PSNR for Projection Layer 1 across 2500 images: 28.95
Average PSNR for Projection Layer 2 across 2500 images: 29.30
Average PSNR for Projection Layer 3 across 2500 images: 29.70
Average PSNR for Projection Layer 4 across 2500 images: 29.79
Average PSNR for Projection Layer 0 across 2500 images: 29.05
Average PSNR for Projection Layer 1 across 2500 images: 29.60
Average PSNR for Projection Layer 2 across 2500 images: 30.41
Average PSNR for Projection Layer 3 across 2500 images: 30.73
Average PSNR for Projection Layer 4 across 2500 images: 31.06
Average PSNR for Projection Layer 0 across 2500 images: 29.87
Average 

In [9]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'DCT_32_Depth10/15 Projections'

DCT_10_proj15_weights, DCT_10_proj15_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj15', display=True)

DCT_20_proj15_weights, DCT_20_proj15_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj15', display=True)

DCT_30_proj15_weights, DCT_30_proj15_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj15', display=True)

DCT_40_proj15_weights, DCT_40_proj15_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, DCT_10_proj15_weights, DCT_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, DCT_20_proj15_weights, DCT_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, DCT_30_proj15_weights, DCT_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, DCT_40_proj15_weights, DCT_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.9162520120938619
Average test loss: 0.005073645493636528
Epoch 4/300
Average training loss: 0.6923293040593466
Average test loss: 0.0056485135037865905
Epoch 5/300
Average training loss: 0.5244792039394379
Average test loss: 0.004802335941750143
Epoch 6/300
Average training loss: 0.40158435424168903
Average test loss: 0.004699764423072338
Epoch 7/300
Average training loss: 0.32146082401275633
Average test loss: 0.004624007799973091
Epoch 8/300
Average training loss: 0.2658291725450092
Average test loss: 0.004583272169240647
Epoch 9/300
Average training loss: 0.2235757715702057
Average test loss: 0.00454707964675294
Epoch 10/300
Average training loss: 0.19063143848048317
Average test loss: 0.004502223805007008
Epoch 11/300
Average training loss: 0.1650684353378084
Average test loss: 0.004570283851275841
Epoch 12/300
Average training loss: 0.14462601422601276
Average test loss: 0.0045452773835923934
Epoch 13/30

In [10]:
DCT_10_proj15_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj15_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj15_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj15_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj15_psnr = average_PSNR(DCT_10_proj15_model, DCT_10_proj15_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj15_psnr = average_PSNR(DCT_20_proj15_model, DCT_20_proj15_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj15_psnr = average_PSNR(DCT_30_proj15_model, DCT_30_proj15_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj15_psnr = average_PSNR(DCT_40_proj15_model, DCT_40_proj15_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj15_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.28
Average PSNR for Projection Layer 1 across 2500 images: 27.71
Average PSNR for Projection Layer 2 across 2500 images: 27.79
Average PSNR for Projection Layer 3 across 2500 images: 27.79
Average PSNR for Projection Layer 4 across 2500 images: 27.94
Average PSNR for Projection Layer 5 across 2500 images: 28.19
Average PSNR for Projection Layer 6 across 2500 images: 28.36
Average PSNR for Projection Layer 7 across 2500 images: 28.12
Average PSNR for Projection Layer 8 across 2500 images: 28.41
Average PSNR for Projection Layer 9 across 2500 images: 28.40
Average PSNR for Projection Layer 10 across 2500 images: 28.38
Average PSNR for Projection Layer 11 across 2500 images: 28.50
Average PSNR for Projection Layer 12 across 2500 images: 28.54
Average PSNR for Projection Layer 13 across 2500 images: 28.64
Average PSNR for Projection Layer 14 across 2500 images: 28.73
Average PSNR for Projection Layer 0 across 2500 images: 28.01
Ave

In [ ]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'DCT_32_Depth10/30 Projections'

DCT_10_proj30_weights, DCT_10_proj30_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj30', display=True)

DCT_20_proj30_weights, DCT_20_proj30_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj30', display=True)

DCT_30_proj30_weights, DCT_30_proj30_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj30', display=True)

DCT_40_proj30_weights, DCT_40_proj30_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, DCT_10_proj30_weights, DCT_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, DCT_20_proj30_weights, DCT_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, DCT_30_proj30_weights, DCT_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, DCT_40_proj30_weights, DCT_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 33.73258789401584
Average test loss: 0.009058227111895878
Epoch 2/300
Average training loss: 20.541846181233723
Average test loss: 0.0058870961483981875
Epoch 3/300
Average training loss: 16.317973705715602
Average test loss: 0.005465657313457794
Epoch 4/300
Average training loss: 12.611602012634277
Average test loss: 0.0051335931556920214
Epoch 5/300
Average training loss: 10.746257635328504
Average test loss: 0.005066527616232634
Epoch 6/300
Average training loss: 8.559011999342177
Average test loss: 0.005285888243052694
Epoch 7/300
Average training loss: 7.769874602423774
Average test loss: 0.013173652932047844
Epoch 8/300
Average training loss: 7.101314699808756
Average test loss: 0.0047027057117472095
Epoch 9/300
Average training loss: 6.096374676174587
Average test loss: 0.00788541219300694
Epoch 10/300
Average training loss: 4.634540271335178
Average test loss: 0.00891591252055433
Epoch 11/300
Average tr

In [ ]:
DCT_10_proj30_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj30_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj30_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj30_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj30_psnr = average_PSNR(DCT_10_proj30_model, DCT_10_proj30_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj30_psnr = average_PSNR(DCT_20_proj30_model, DCT_20_proj30_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj30_psnr = average_PSNR(DCT_30_proj30_model, DCT_30_proj30_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj30_psnr = average_PSNR(DCT_40_proj30_model, DCT_40_proj30_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj30_psnr, f)